## Importing Libraries

In [ ]:
import argparse
import multiprocessing
import os

import torch
import transformers
from accelerate import PartialState
from datasets import load_dataset
from peft import LoraConfig
from transformers import (
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    logging,
    set_seed,
)
from trl import SFTTrainer


## Testing DeepSeek Loading

In [1]:

# config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)
lora_config = LoraConfig(
    r=8,
    target_modules=[
        "q_proj",
        "o_proj",
        "k_proj",
        "v_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    task_type="CAUSAL_LM",
)
# load model and dataset
token = os.environ.get("HF_TOKEN", None)
# print out selected device
print(PartialState().process_index)
#Check for GPU availability
print(torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU")
    with torch.device("cuda:2"):
        model = AutoModelForCausalLM.from_pretrained(
            "deepseek-ai/DeepSeek-Coder-V2-Lite-Instruct",
            quantization_config=bnb_config,
            attention_dropout=0.1,
            trust_remote_code=True,
        ).to("cuda:2")
else:
    #Handle no GPU availiability
    print("No GPU")
    with torch.device("cpu"):
        model = AutoModelForCausalLM.from_pretrained(
            "deepseek-ai/DeepSeek-Coder-V2-Lite-Instruct",
            quantization_config=bnb_config,
            attention_dropout=0.1,
            trust_remote_code=True,
        ).to("cpu")

#if torch.cuda.device_count() > 1:
#    print("Using DataParallel for multiple GPUs")
#    model = torch.nn.DataParallel(model)

#model.to(device)
# Freeze all except embeddings and first layer
#for name, param in model.named_parameters():
#    if "model.embed_tokens" not in name and "model.layers.0" not in name:
#        param.requires_grad = False
#    else:
#        param.requries_grad = True

/media/mujtaba/DATA/nick/miniconda3/envs/UnitTestGeneration/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0
True
GPU


`low_cpu_mem_usage` was None, now default to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 4/4 [00:54<00:00, 13.74s/it]


## Testing Dataset Training

In [ ]:
print(args.dataset_name)
    print(args.subset)
    print(args.split)
    print(token)
    print(args.num_proc if args.num_proc else multiprocessing.cpu_count())
    
    '''Original file from hugging face hub
    data = load_dataset(
        args.dataset_name,
        data_files=args.subset,
        split=args.split,
        token=token,
        num_proc=args.num_proc if args.num_proc else multiprocessing.cpu_count(),
    )'''

    data = load_dataset(
        'parquet', 
        data_files=args.subset, 
        split=args.split,
        num_proc=args.num_proc if args.num_proc else multiprocessing.cpu_count(),
    )

    print('Dataset type: ', type(data))